# Setup Instructions

## 1. Set Your API Key

Open the `.env` file provided in the repository and replace `your_api_key_here` with your actual Gemini API key:

```
GEMINI_API_KEY=your_api_key_here
```

## 2. How to Get a Gemini API Key

1. Go to [Google AI Studio](https://aistudio.google.com)
2. Sign in with your Google account
3. Click **"Get API Key"** in the sidebar
4. Click **"Create API key"** → Select **"Create API key in new project"**
5. Copy your API key (it starts with `AIza...`)

## 3. Install Dependencies

```bash
pip install -q -U google-genai python-dotenv
pip install python-dotenv
```

## 4. Run the Notebook

You're all set! Run the cells to extract text from menu images.

In [ ]:
# install pip dependencies
pip install -q -U google-genai
pip install python-dotenv

In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import json

# Initialize client
load_dotenv()

# Client automatically uses GEMINI_API_KEY from environment
client = genai.Client()
# client = genai.Client(api_key=)  # Or set GEMINI_API_KEY env var


def extract_text_from_image(image_path: str) -> str:
    """Extract all text from a menu image."""
    
    # Read image bytes
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
    
    # Determine mime type
    mime_type = "image/jpeg"
    if image_path.lower().endswith(".png"):
        mime_type = "image/png"
    elif image_path.lower().endswith(".webp"):
        mime_type = "image/webp"
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
            """Extract ALL text from this menu image.
            Preserve the structure and formatting as much as possible.
            Include prices, descriptions, and any dietary symbols/notes."""
        ]
    )
    
    return response.text


def extract_as_structured_json(image_path: str) -> dict:
    """Extract menu as structured JSON."""
    
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
    
    mime_type = "image/jpeg"
    if image_path.lower().endswith(".png"):
        mime_type = "image/png"
    
    # Use JSON response mode for cleaner output
    config = types.GenerateContentConfig(
        response_mime_type="application/json"
    )
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
            """Extract the menu into structured JSON format:
            {
                "restaurant_name": "Name if visible",
                "sections": [
                    {
                        "name": "Section Name",
                        "items": [
                            {
                                "name": "Item Name",
                                "price": "$X.XX",
                                "description": "Description text",
                                "dietary_info": ["vegetarian", "gluten-free", etc]
                            }
                        ]
                    }
                ]
            }"""
        ],
        config=config
    )
    
    return json.loads(response.text)


# Usage
if __name__ == "__main__":
    # Basic text extraction
    text = extract_text_from_image("./tatte_menu.JPG")
    print(text)
    
    # Structured JSON extraction
    # data = extract_as_structured_json("menu.png")
    # print(json.dumps(data, indent=2))

BRUNCH
Served all day Friday, Saturday, and Sunday
TATTE
HOMEMADE
FRESH

— Breakfast —

**BREAKFAST SANDWICH*** - $9.85
Eggs your style*, VT cheddar, and applewood
smoked bacon on housemade sourdough. ⓌⒹⒺⒼ

**CROISSANT BREAKFAST SANDWICH*** - $11.25
Egg your style*, VT cheddar, sliced tomato, avocado,
baby arugula on housemade croissant. ⓌⒹⒺⒼ

**HALLOUMI SUNNY-SIDE BREAKFAST SANDWICH*** - $9.85
Seared halloumi cheese, griddled tomato, sautéed
spinach with sunny-side up egg* on housemade
challah roll. ⓌⒹⒺⒼⓋ

**BISCUIT & EGG SANDWICH** - $9.50
Two eggs soft scrambled served on a housemade
biscuit served with tomato salad. ⓌⒹⒺⓋ

**SMOKED SALMON, AVOCADO, & EGG SANDWICH** - $12.50
Smoked salmon, avocado, red onion, capers, alfalfa
sprouts, and creamy scrambled eggs served on housemade
challah roll with green herbed dressing. ⓌⒹⒺⒻⓈ

**LAMB HASH*** - $15.25
Our version of hash. Lamb cooked in warm spices
sautéed with potatoes, sweet potatoes, carrots, and
pickled red cabbage topped with poac